Graph Distance
=

This notebook explores quantitative methods of assessing the distance between two graphs, with the goal of measuring the consistency of scene graphs between frames.

# NetworkX Edit Distance

NetworkX directional graph
- All objects are nodes
    - Bounding box as an attribute
    - Category is an attribute
    - Leave attributes as an attribute in the object
~~- All attributes are nodes (!!) - Draw directed edge from object to attribute? is that too heavy~~
- Predicates
    - Directed graph from obj1 to obj2
    - Predicate name is the edge's attribute


Edit distance logistics
- predicate edge must have the same label
- objects must have the same category and iou > X%
- attributes?
    - we can change the cost of different operations. but the problem is we'd want predicate edges to be worth more than attribute edges?

In [ ]:
import networkx as nx
import json

In [3]:
json_graphs = []
with open('graphs/shortvid_scene_graphs.json') as f:
    json_graphs = json.load(f)


graphs = []
for i, graph in enumerate(json_graphs):
    g = nx.DiGraph()
    # add all objects
    for id, obj in graph['objects'].items():
        g.add_node(id, **obj) # category, bounding_box, attributes
    
    # add all relationships
    for obj1, obj2, pred in graph['relations']:
        g.add_edge(obj1, obj2, predicate=pred)

    graphs.append(g)

In [6]:
graphs[0]['chair_1']['table_1']

{'predicate': 40}

In [7]:
# define fns for edit distance algorithm

def iou(bb1, bb2):
    x11, y11, x12, y12 = bb1
    x21, y21, x22, y22 = bb2
    x_overlap = max(0, min(x12, x22) - max(x11, x21))
    y_overlap = max(0, min(y12, y22) - max(y11, y21))
    intersection = x_overlap * y_overlap
    union = (x12 - x11) * (y12 - y11) + (x22 - x21) * (y22 - y21) - intersection
    iou = intersection / union
    return iou

def node_match(n1, n2):
    if n1['category'] != n2['category']:
        return False

    return iou(n1['bounding_box'], n2['bounding_box']) >= 0.8

def edge_match(e1, e2):
    return e1['predicate'] == e2['predicate']

def node_subst_cost(n1, n2):
    cost = int(node_match(n1, n2))
    # add 0.1 for every differing attribute
    diff_attrs = set(n1['attributes']) ^ set(n2['attributes']) # shorthand for symmetric difference
    cost += 0.1 * len(diff_attrs)
    return cost

In [8]:
def diff(g1, g2):
    return nx.graph_edit_distance(g1, g2, node_match=node_match, edge_match=edge_match, node_subst_cost=node_subst_cost)

In [ ]:
print(diff(graphs[0], graphs[0]))

In [10]:
print(diff(graphs[0], graphs[1]))

KeyboardInterrupt: 